### Imports

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mizo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import sklearn
import email.parser
import os
import copy

# corpus preprocessing

### load dataset

In [3]:
tr_src_dir = input('Input source directory TR: ')

Input source directory TR: C:\Users\Mizo\Documents\ICYBM121-p2\data\TR


In [4]:
tt_src_dir = input('Input source directory TT: ')

Input source directory TT: C:\Users\Mizo\Documents\ICYBM121-p2\data\TT


In [5]:
dest_dir = input('Input destination directory: ')

Input destination directory: C:\Users\Mizo\Documents\ICYBM121-p2


### lexical analysis (tokenization)
headers, attachments and HTML tags are stripped
result only contains the e-mail body and subject line

In [6]:
def lexical_analysis(mail):
    m = open(mail, 'rb')
    try:
        msg = email.message_from_binary_file(m)
        subject = msg.get('subject')
        multipart_counter = 0
        no_txt = True
        while no_txt and msg.is_multipart():
            
            msg = msg.get_payload(multipart_counter)
            if isinstance(msg.get_payload(), str):
                no_txt = False 
            multipart_counter += 1
        body = msg.get_payload()
    except Exception as e:
        print(e)
        print('error parsing mail')
        body = 'not available'
        subject = 'not available'
        
        
    return subject, body

### extract field from mail

In [ ]:
def get_field_of_mail(mail,field):
    m = open(mail, 'rb')
    try:
        msg = email.message_from_binary_file(m)
        extracted_field = msg.get(field)
    except Exception as e:
        extracted_field = 'not available'
        
    return extracted_field

### stop-word removal

In [7]:
def stop_word_removal(text):
    new_text = text
    # list of english stopwords
    stop_words = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thick", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]
    for sw in stop_words:
        if sw in text.lower():
            word = ' ' + sw + ' '
            new_text = text.lower().replace(word, ' ')
        text = new_text
    return new_text

### stemming
source: https://www.geeksforgeeks.org/python-stemming-words-with-nltk/

In [8]:
 def stemming(text):
    new_text = ''
    ps = PorterStemmer()
    words = word_tokenize(text)
     
    for w in words: 
        new_text += ps.stem(w) + ' ' 
    return new_text

### non-word removal

In [9]:
def non_word_removal(text):
    new_text = ''
    words = word_tokenize(text)
    
    for word in words:
        if word.isalpha():
            new_text += word + ' '

    return new_text        
        

# Algorithm 1. Content Based Spam E-mail Filtering 
by Liu et Moh, San Jose State University

### Imports

In [ ]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import re

### remove HTML tags
source: https://medium.com/@jorlugaqui/how-to-strip-html-tags-from-a-string-in-python-7cb81a2bbf44

In [13]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

### update word frequency in given dictionary

In [14]:
def update_frequency_count(text, class_dict):
    words = word_tokenize(text)
    for word in words:
        w = word.lower()
        if w in class_dict.keys():
            class_dict[w] += 1
        else:
            class_dict[w] = 1
    return class_dict
        
    

### Look up the classification of the given mail of the training set

In [15]:
def lookup_classification_for_training_set(mail):
    classification = ''
    number = mail.replace('TRAIN_', '').replace('.eml','')
    with open('./data/spam-mail.tr.label') as lf:
        lines = lf.readlines()
        for line in lines:
            [mail_nr, c] = line.split(',')
            if mail_nr == number:
                classification = c
    return classification
        

### Training phase

In [77]:
 def training_algorithm_1(tr_src_dir):
    spam_dict = {}
    ham_dict = {}
    if os.path.isdir(tr_src_dir):
        for mail in os.listdir(tr_src_dir):
            mail_path = '{}/{}'.format(tr_src_dir, mail)
            _, body = lexical_analysis(mail_path)
            if not isinstance(body,list):
                body = remove_html_tags(body)
                body = stemming(body)
                body = non_word_removal(body)
            
                classification = lookup_classification_for_training_set(mail)
                classification = classification.replace('\n', '')
                if int(classification) == 0:
                    spam_dict = update_frequency_count(body, spam_dict)
                else:
                    ham_dict = update_frequency_count(body, ham_dict)

    
    threshold = 0
    spam_dict_Copy = copy.deepcopy(spam_dict)
    for word in spam_dict:
        count = spam_dict[word]
        if count < threshold:
            del spam_dict_Copy[word]
    
    ham_dict_Copy = copy.deepcopy(ham_dict)
    for word in ham_dict:
        count = ham_dict[word]
        if count < threshold:
            del ham_dict_Copy[word]
    
    return spam_dict_Copy, ham_dict_Copy
            
            

### Testing phase

In [71]:
def testing_algorithm_1(spam_dict, ham_dict, threshold, tt_src_dir):
    spam_num = 779
    ham_num = 1721
    fd = open('spam-mail.tt.label', 'w')
    fd.write('Id,Prediction')
    fd.close()
    if os.path.isdir(tt_src_dir):
        for mail in os.listdir(tt_src_dir):
            classification = 1
            mail_path = '{}/{}'.format(tt_src_dir, mail)
            _, body = lexical_analysis(mail_path)
            if not isinstance(body,list):
                body = remove_html_tags(body)
                body = stemming(body)
                body = non_word_removal(body)
                prob_spam = 1.0
                prob_ham = 1.0
                words = word_tokenize(body)
                for word in words:
                    w = word.lower()
                    if w in spam_dict.keys() and w in ham_dict.keys():
                        spam_count = spam_dict[w]
                        ham_count = ham_dict[w]
                        p1 = (spam_count / spam_num) / ((spam_count / spam_num) + (ham_count / ham_num))
                        p2 = 1 - p1
                        prob_spam = prob_spam * p1
                        prob_ham = prob_ham * p2
                if prob_spam - prob_ham > threshold:
                    classification = 0
                else:
                    classification = 1
            email_id = mail.replace('TEST_', '').replace('.eml','')
            fd = open('spam-mail.tt.label', 'a')
            fd.write('\n%d,%d' % (int(email_id), int(classification)))
            fd.close()
            
                
    

### Run algorithm

In [81]:
spam_dict, ham_dict = training_algorithm_1(tr_src_dir)
testing_algorithm_1(spam_dict, ham_dict, 0, tt_src_dir)

list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail


# Algorithm 2. Spam filtering by anomaly detection

### Imports

import numpy as np
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_extraction.text import TfidfVectorizer

### Training phase

In [48]:
def training_algorithm_2(tr_src_dir):
    train_set = []
    if os.path.isdir(tr_src_dir):
        for mail in os.listdir(tr_src_dir):
            mail_path = '{}/{}'.format(tr_src_dir, mail)
            
            #preprocessing of mail
            #only use body
            body = lexical_analysis(mail_path)
            content = str(body)
                
            train_set.append(content)
            
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(train_set)
        
        clf = LocalOutlierFactor(n_neighbors=20)
        clf.fit(X)
        
    return clf #LOF
            
            
            
    

### Testing phase

In [49]:
def testing_algorithm_2(tt_src_dir, LOF):
    fd = open('spam-mail.tt.LOF.label', 'w')
    fd.write('Id,Prediction')
    fd.close()
    test_set = []
    if os.path.isdir(tt_src_dir):
        for mail in os.listdir(tt_src_dir):
            mail_path = '{}/{}'.format(tt_src_dir, mail)
            
            #preprocessing of mail
            #only use body
            _, body = lexical_analysis(mail_path)
            content = str(body)
                
            test_set.append(content)
        
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(test_set)  
        
        res = LOF.fit_predict(X)
        
        fd = open('spam-mail.tt.LOF.label', 'a')
        for i in range(len(res)):
            if res[i] == -1:
                fd.write('\n%d,%d' % ( i+1 , 0))
            else :
                fd.write('\n%d,%d' % ( i+1 , 1))
        fd.close()
        return res
   
            

### Run algorithm

In [50]:
LOF = training_algorithm_2(tr_src_dir)
res = testing_algorithm_2(tt_src_dir, LOF)
print(len(res))
unique, counts = np.unique(res, return_counts=True)

print(dict(zip(unique, counts)))


list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail


C:\Users\Mizo\Anaconda3\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


list index out of range
error parsing mail
list index out of range
error parsing mail
list index out of range
error parsing mail


C:\Users\Mizo\Anaconda3\lib\site-packages\sklearn\neighbors\lof.py:236: FutureWarning: default contamination parameter 0.1 will change in version 0.22 to "auto". This will change the predict method behavior.
  FutureWarning)


1827
{-1: 183, 1: 1644}


# Experimenting

### Preprocessing of mails

In [10]:
def preprocess_mails(src_dir, dest_dir):
    if os.path.isdir(src_dir):
        for mail in os.listdir(src_dir):
            mail_path = '{}/{}'.format(src_dir, mail)
            print(mail_path)
            subject, body = lexical_analysis(mail_path)
            print(body)
    else:
        print('no source dir')
            

### SVM 

In [18]:
from sklearn.svm import SVC
def svmTest(tr_src_dir, tt_src_dir):
    dico, classificationList = prepare_Dico(tr_src_dir)
    TR_matrix = features_matrix(tr_src_dir, dico, 2500) 
    clf = SVC()
    print(len(TR_matrix))
    print(len(classificationList))
    clf.fit(TR_matrix , classificationList)
    
    #dico, classificationList = prepare_Dico(tt_src_dir, False)
    TT_matrix = features_matrix(tt_src_dir, dico, 1827) 
    res = clf.predict(TT_matrix)
    print(np.count_nonzero(res))
    print("res size : ", len(res))
    fd = open('spam-mail.tt.label', 'a')
    for i in range(len(res)):
        
        fd.write('\n%d,%d' % ( i+1 , res[i]))
    fd.close()

    return res

In [16]:
def prepare_Dico(tr_src_dir, doClassificationList = True):
    dico = {}
    classificationList = []
    if os.path.isdir(tr_src_dir):
        for mail in os.listdir(tr_src_dir):
            mail_path = '{}/{}'.format(tr_src_dir, mail)
            _, body = lexical_analysis(mail_path)
            if not isinstance(body,list):
                body = remove_html_tags(body)
                body = stemming(body)
                body = non_word_removal(body)
                dico = update_frequency_count(body, dico)
            if doClassificationList:
                classification = lookup_classification_for_training_set(mail)
                classification = classification.replace('\n', '')
                if int(classification) == 0:
                    classificationList.append(0)
                else:
                    classificationList.append(1)
    threshold = 10
    dicoCopy = copy.deepcopy(dico)
    print("dico size before : ", len(dico))
    
    for word in dico:
        count = dico[word]
        if count < threshold:
            del dicoCopy[word]
    
    print("dico size after : ", len(dicoCopy))
    return dicoCopy, classificationList

In [17]:
def features_matrix(tr_src_dir, dico, size):
    id = 0
    
    matrix = np.zeros((size,len(dico)))
    enumerateDico = []
    for i, j in enumerate(dico):
        enumerateDico.append((i,j))
    for mail in os.listdir(tr_src_dir):
        mail_path = '{}/{}'.format(tr_src_dir, mail)
        _, body = lexical_analysis(mail_path)
        if not isinstance(body,list):
            body = body.split()
            for word in body:
                for i, j in enumerateDico:
                    if str(j) == word:
                        matrix[id,i] += 1 
        
            
        id += 1
    print("id :",id )
    print("count :",np.count_nonzero(matrix))
    return matrix

In [72]:
svmTest(tr_src_dir, tt_src_dir)
dico, classification = prepare_Dico(tr_src_dir)
features_matrix(tr_src_dir, dico, 2145) 
spam_dict, ham_dict, threshold = training_algorithm_1(tr_src_dir)
print(threshold)